In [ ]:
from PIL import Image
import heapq


class HuffmanNode:
    def __init__(self, pixel=None, freq=0):
        self.pixel = pixel
        self.freq = freq
        self.left = None
        self.right = None

    def __lt__(self, other):
        return self.freq < other.freq


def calculate_frequency(image):
    frequency = {}
    width, height = image.size
    for y in range(height):
        for x in range(width):
            pixel = image.getpixel((x, y))
            if pixel in frequency:
                frequency[pixel] += 1
            else:
                frequency[pixel] = 1
    return frequency


def build_huffman_tree(frequency):
    heap = [HuffmanNode(pixel, freq) for pixel, freq in frequency.items()]
    heapq.heapify(heap)

    while len(heap) > 1:
        left = heapq.heappop(heap)
        right = heapq.heappop(heap)
        parent = HuffmanNode(freq=left.freq + right.freq)
        parent.left = left
        parent.right = right
        heapq.heappush(heap, parent)

    return heap[0]


def generate_huffman_codes(node, code, huffman_codes):
    if node.pixel is not None:
        huffman_codes[node.pixel] = code
    else:
        generate_huffman_codes(node.left, code + "0", huffman_codes)
        generate_huffman_codes(node.right, code + "1", huffman_codes)


def compress_frame(frame):
    image = Image.fromarray(frame)
    frequency = calculate_frequency(image)
    huffman_tree = build_huffman_tree(frequency)
    huffman_codes = {}
    generate_huffman_codes(huffman_tree, "", huffman_codes)

    # Compress the image data
    compressed_data = ""
    width, height = image.size
    for y in range(height):
        for x in range(width):
            pixel = image.getpixel((x, y))
            compressed_data += huffman_codes[pixel]

    # Pad the compressed data to ensure it can be divided into 8-bit chunks
    padding_length = 8 - (len(compressed_data) % 8)
    compressed_data += "0" * padding_length

    # Convert the compressed data to bytes
    compressed_bytes = bytearray()
    for i in range(0, len(compressed_data), 8):
        byte = compressed_data[i:i+8]
        compressed_bytes.append(int(byte, 2))

    return compressed_bytes


In [ ]:
import cv2

def compress_video(input_path, output_path):
    # Open the input video file
    video = cv2.VideoCapture(input_path)

    # Get video properties
    frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(video.get(cv2.CAP_PROP_FPS))

    # Create a video writer object
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")  # Codec for MP4 format
    output_video = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    # Iterate through each frame of the video
    while video.isOpened():
        ret, frame = video.read()

        if not ret:
            break

        # Compress the frame
        compressed_frame = compress_frame(frame)

        # Write the compressed frame to the output video
        output_video.write(compressed_frame)

    # Release video objects
    video.release()
    output_video.release()

    print("Video compression complete.")

# Usage
input_path = "input_video.mp4"
output_path = "compressed_video.mp4"
compress_video(input_path, output_path)


Стиснення відео передбачає стиснення кожного кадру відео за допомогою алгоритму стиснення зображення, такого як згаданий раніше (наприклад, кодування Хаффмана), а потім кодування стиснутих кадрів у відеоформат. Ось загальний огляд процесу:

Завантажте відео: використовуйте бібліотеку обробки відео, як-от OpenCV або moviepy, щоб завантажити відеофайл.
Вилучення кадрів: переглядайте кожен кадр відео та витягуйте окремі кадри як зображення.
Застосувати стиснення зображення: застосувати вибраний алгоритм стиснення зображення (наприклад, кодування Хаффмана), щоб стиснути кожен кадр окремо, дотримуючись кроків, згаданих раніше для стиснення зображення.
Кодувати стислі кадри: використовуйте бібліотеку кодування відео, таку як FFmpeg або OpenCV, щоб кодувати стислі кадри у відеоформат (наприклад, MP4, AVI).
Зберегти стиснене відео: зберегти закодований відеофайл у потрібному місці.
Майте на увазі, що стиснення відео передбачає багато додаткових міркувань, таких як методи міжкадрового стиснення (наприклад, оцінка руху, часова компресія) і вибір відповідних відеокодеків. Описаний тут процес забезпечує базове розуміння стиснення відео за допомогою методів стиснення зображення.

Ось спрощений приклад, який демонструє стиснення відео за допомогою бібліотеки OpenCV:

У цьому прикладі функція compress_video приймає вхідний відеошлях і вихідний відеошлях як параметри. Він використовує об’єкт cv2.VideoCapture для читання кадрів із вхідного відео та об’єкт cv2.VideoWriter для запису стислих кадрів у вихідне відео. Функція compress_frame (розглянута раніше) використовується для стиснення кожного окремого кадру перед записом його у вихідне відео.

Обов’язково налаштуйте кодек (fourcc) відповідно до бажаного формату вихідного відео. Крім того, вам може знадобитися інсталювати необхідні бібліотеки (OpenCV і Pillow), якщо ви цього ще не зробили.

Майте на увазі, що цей приклад демонструє базову реалізацію стиснення відео з використанням техніки стиснення зображення. Стиснення відео в реальному світі вимагає додаткових міркувань, таких як обробка ключових кадрів, застосування методів тимчасового стиснення та вибір відповідних кодеків для оптимального стиснення.

Щоб стиснути зображення в Python за допомогою таких алгоритмів, як Law або Huffman, виконайте такі загальні дії:

Завантажте зображення. Почніть із завантаження зображення за допомогою бібліотеки, наприклад Pillow або OpenCV.
Попередня обробка зображення: конвертуйте зображення в потрібний формат (наприклад, у градаціях сірого) і за потреби відрегулюйте розмір.
Застосувати алгоритм стиснення: запровадити вибраний алгоритм стиснення, наприклад, кодування Лоу або Хаффмана.
a. Алгоритм Закону: Алгоритм Закону — це проста техніка стиснення, яка використовує попередньо визначений набір математичних функцій для представлення даних зображення. Вам потрібно буде визначити набір базових функцій і розкласти зображення на коефіцієнти за допомогою цих функцій. Потім ці коефіцієнти можна кодувати та зберігати більш ефективно.
b. Кодування Хаффмана: кодування Хаффмана — це алгоритм стиснення без втрат, який призначає коротші коди значенням даних, що зустрічаються частіше. Вам потрібно буде обчислити частоту різних значень пікселів на зображенні та створити дерево Хаффмана. Використовуючи це дерево, ви можете призначати коди змінної довжини значенням пікселів і замінювати вихідні значення пікселів відповідними кодами.
Збережіть стиснуті дані: збережіть стиснуті дані у файл разом із будь-якою необхідною інформацією для розпакування.
Ось спрощений приклад, який демонструє кодування Хаффмана для стиснення зображення за допомогою бібліотеки Pillow:

In [ ]:
from PIL import Image
import heapq
import os


class HuffmanNode:
    def __init__(self, pixel=None, freq=0):
        self.pixel = pixel
        self.freq = freq
        self.left = None
        self.right = None

    def __lt__(self, other):
        return self.freq < other.freq


def calculate_frequency(image):
    frequency = {}
    width, height = image.size
    for y in range(height):
        for x in range(width):
            pixel = image.getpixel((x, y))
            if pixel in frequency:
                frequency[pixel] += 1
            else:
                frequency[pixel] = 1
    return frequency


def build_huffman_tree(frequency):
    heap = [HuffmanNode(pixel, freq) for pixel, freq in frequency.items()]
    heapq.heapify(heap)

    while len(heap) > 1:
        left = heapq.heappop(heap)
        right = heapq.heappop(heap)
        parent = HuffmanNode(freq=left.freq + right.freq)
        parent.left = left
        parent.right = right
        heapq.heappush(heap, parent)

    return heap[0]


def generate_huffman_codes(node, code, huffman_codes):
    if node.pixel is not None:
        huffman_codes[node.pixel] = code
    else:
        generate_huffman_codes(node.left, code + "0", huffman_codes)
        generate_huffman_codes(node.right, code + "1", huffman_codes)


def compress_image(image_path, output_path):
    image = Image.open(image_path)
    frequency = calculate_frequency(image)
    huffman_tree = build_huffman_tree(frequency)
    huffman_codes = {}
    generate_huffman_codes(huffman_tree, "", huffman_codes)

    # Compress the image data
    compressed_data = ""
    width, height = image.size
    for y in range(height):
        for x in range(width):
            pixel = image.getpixel((x, y))
            compressed_data += huffman_codes[pixel]

    # Pad the compressed data to ensure it can be divided into 8-bit chunks
    padding_length = 8 - (len(compressed_data) % 8)
    compressed_data += "0" * padding_length

    # Convert the compressed data to bytes
    compressed_bytes = bytearray()
    for i in range(0, len(compressed_data), 8):
        byte = compressed
